In [1]:
import torch
import torchvision
import numpy as np

from benchmark import ImageBenchmark

In [2]:
bench = ImageBenchmark()
models = list(bench.list_models())
for i, model in enumerate(models):
    if not model.torch_model_exists():
        continue
    print(f'{i}\t {model.__str__()}')

0	 pretrain(mbnetv2,ImageNet)-
1	 pretrain(resnet18,ImageNet)-
8	 pretrain(mbnetv2,ImageNet)-transfer(MIT67,0.1)-
9	 pretrain(mbnetv2,ImageNet)-transfer(MIT67,0.5)-
10	 pretrain(mbnetv2,ImageNet)-transfer(MIT67,1)-
11	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.1)-
12	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,0.5)-
13	 pretrain(mbnetv2,ImageNet)-transfer(Flower102,1)-
14	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.1)-
15	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,0.5)-
16	 pretrain(mbnetv2,ImageNet)-transfer(SDog120,1)-
17	 pretrain(resnet18,ImageNet)-transfer(MIT67,0.1)-
18	 pretrain(resnet18,ImageNet)-transfer(MIT67,0.5)-
19	 pretrain(resnet18,ImageNet)-transfer(MIT67,1)-
20	 pretrain(resnet18,ImageNet)-transfer(Flower102,0.1)-
21	 pretrain(resnet18,ImageNet)-transfer(Flower102,0.5)-
22	 pretrain(resnet18,ImageNet)-transfer(Flower102,1)-
23	 pretrain(resnet18,ImageNet)-transfer(SDog120,0.1)-
24	 pretrain(resnet18,ImageNet)-transfer(SDog120,0.5)-
25	 pretrain(resnet18,I

In [3]:
from modeldiff import ModelDiff

def compare_with_seed(model1, model2, truth=-1):
    print(f'comparing:\n  model1={model1}\n  model2={model2}')
    md = ModelDiff(model1, model2)
    seed_inputs = md.get_seed_inputs(rand=False)
    sim = md._compute_distance(seed_inputs)
    if truth == -1:
        truth = 1 if model1.__str__().split('-')[0] == model2.__str__().split('-')[0] else 0
    print(f' similarity is {sim}, truth is {truth}')

def test(compare):
    compare(models[1], models[0], 0)   # should be different
    compare(models[13], models[25], 0) # should be different
    compare(models[13], models[22], 0) # should be different
    compare(models[1], models[17], 1)  # should be similar
    compare(models[16], models[13], 1) # should be similar
    compare(models[13], models[12], 1) # should be similar
    
# test(compare_with_seed)

In [ ]:
DEVICE = 'cuda'

def gen_adv_inputs(model, inputs):
    from advertorch.attacks import LinfPGDAttack
    def myloss(yhat, y):
        return -((yhat[:,0]-y[:,0])**2 + 0.1*((yhat[:,1:]-y[:,1:])**2).mean(1)).mean()
        
    model = model.to(DEVICE)
    inputs = torch.from_numpy(inputs).to(DEVICE)
    with torch.no_grad():
        model.eval()
        clean_outputs = model(inputs)
    
    output_shape = clean_outputs.shape
    batch_size = output_shape[0]
    num_classes = output_shape[1]
    y = torch.zeros(size=output_shape).to(DEVICE)
    y[:, 0] = 1000
#     # more diversity
#     rand_idx = torch.randint(low=0, high=num_classes, size=(batch_size,))
#     y = torch.nn.functional.one_hot(rand_idx, num_classes=num_classes).to(DEVICE) * 1000
#     print(y)
    
    adversary = LinfPGDAttack(
        model, loss_fn=myloss, eps=0.1,
        nb_iter=40, eps_iter=0.01, 
        rand_init=True, clip_min=-2.2, clip_max=2.2,
        targeted=False
    )
    
    adv_inputs = adversary.perturb(inputs, y)
    
    with torch.no_grad():
        model.eval()
        adv_outputs = model(adv_inputs).to('cpu').numpy()
#     print(adv_outputs)
    torch.cuda.empty_cache()
    return adv_inputs.to('cpu').numpy()


def compare_with_adv(model1, model2, truth=-1):
    print(f'comparing:\n  model1={model1}\n  model2={model2}')
    md = ModelDiff(model1, model2)
    rand = False
    seed_inputs1 = model1.get_seed_inputs(100, rand=rand)
    seed_inputs2 = model2.get_seed_inputs(100, rand=rand)
    seed_inputs = np.concatenate([seed_inputs1, seed_inputs2])
    
    adv_inputs1 = gen_adv_inputs(model1.torch_model, seed_inputs1)
    adv_inputs2 = gen_adv_inputs(model2.torch_model, seed_inputs2)
    adv_inputs = np.concatenate([adv_inputs1, adv_inputs2])
    
    adv_inputs_shuffle = list(adv_inputs)
    np.random.shuffle(adv_inputs_shuffle)
    adv_inputs_shuffle = np.array(adv_inputs_shuffle)
    
    sim = md._compute_distance(adv_inputs_shuffle)
    if truth == -1:
        truth = 1 if model1.__str__().split('-')[0] == model2.__str__().split('-')[0] else 0
    print(f' similarity is {sim}, truth is {truth}')
    batch_outputs1 = model1.batch_forward(adv_inputs)
    _, batch_preds1 = batch_outputs1.to('cpu').data.max(1)
    batch_outputs2 = model2.batch_forward(adv_inputs)
    _, batch_preds2 = batch_outputs2.to('cpu').data.max(1)
    print(f'  batch_preds1={batch_preds1}\n  batch_preds2={batch_preds2}')


# compare_with_adv(models[1], models[17], 1)
test(compare_with_adv)

comparing:
  model1=pretrain(resnet18,ImageNet)-
  model2=pretrain(mbnetv2,ImageNet)-
 similarity is 0.9385082105626885, truth is 0
  batch_preds1=tensor([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 179, 925, 925, 825, 293, 912, 817,  31, 563, 241, 202, 925,
        925, 435, 985, 925, 879, 925, 326, 811, 925, 985, 817, 925, 925, 925,
         31, 925, 811, 925, 985, 985, 825, 563, 925, 925, 825, 478, 925, 925,
        925, 310, 825, 925, 925, 720, 925, 563, 825, 925, 811, 199, 705, 

In [5]:
from utils import Utils
from matplotlib import pyplot as plt

def show_images(images, labels, title='examples'):
    images = np.transpose(images, (0, 2, 3, 1))
    plt.figure(figsize=(10,10))
    plt.subplots_adjust(hspace=0.2)
    for n in range(25):
        plt.subplot(5,5,n+1)
        img = images[n]
        img = img.squeeze()
        plt.imshow(img)
        plt.title(f'{labels[n]}')
        plt.axis('off')
    _ = plt.suptitle(title)
    plt.show()

# show_images(seed_inputs, list(range(seed_inputs.shape[0])))
# print(seed_inputs[0])
# batch_outputs1 = model1.batch_forward(seed_inputs)
# batch_preds1 = batch_outputs1.to('cpu').data.max(1)
# print(batch_preds1)

In [6]:
torch.cuda.empty_cache()